# Libraries

In [1]:
%matplotlib inline
import pandas as pd, datetime as dt
#from co2mpas.datasync import _get_interp_method
import collections
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import glob, os
import time

## Definitions

In [2]:
car = {'Audi A1': ['./Audi A1/', 'xls',98,'gasoline'],             # we could get vela2 PN data from the analysis_veh.xls DB
      'BMW 530d': ['./BMW 530d/', 'xlsx',124,'diesel'],
      'Audi A3': ['./Audi A3/', 'xls',120,'diesel'],               # specify the FWD for NEDC in function lab_a1 
      'Citroen Cactus': ['./Citroen Cactus/','xls',95,'diesel'],   # specify WLTP_COLD in function lab_a1
      'Fiat 500x': ['./Fiat 500x/','xls',144,'diesel'],
      'Opel Astra': ['./Opel Astra/','',103,'gasoline'],           # Alternative code because of Vela8 file format (.xls for WLTP and .xlsx for NEDC)
      'Fiat Panda': ['./Fiat Panda/', 'xlsx',119,'gasoline'],
      'Kia Sportage': ['./Kia Sportage/','xlsx',124,'diesel'],
      'Peugeot Partner': ['./Peugeot Partner/','xls',112,'diesel'],
      'Porsche Cayenne': ['./Porsche Cayenne/','xlsx',179,'diesel'],
      'Renault Twingo': ['./Renault Twingo/', 'xlsx',112,'gasoline'],
      'VW Golf': ['./VW Golf/','xlsx',117,'diesel']
      #'VW Tiguan': ['./VW Tiguan/','xlsx',159,'diesel'],          # this remains OUT. not enough info and it'd require changes 
      }

In [3]:
vehicle = 'Porsche Cayenne'

In [4]:
path = car[vehicle][0]
filetype = car[vehicle][1]
t_a = car[vehicle][2]
fuel = car[vehicle][3]

## Create the empty dataframe (ONLY BEFORE THE 1ST CAR!!!)

In [ ]:
"""
###############################
Only to be used the first time!
###############################
"""

arrays = [['fuel','T_A',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'NEDC','NEDC','NEDC','NEDC','NEDC',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'WLTP','WLTP','WLTP','WLTP','WLTP','WLTP','WLTP',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE',
           'RDE','RDE','RDE','RDE','RDE'],
           ['-', '-', 
           'CO2','CO2','CO2','CO2','CO2',             # reorder columns
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN','PN','PN',
           'CO2','CO2','CO2','CO2','CO2',  
           'NOx','NOx','NOx','NOx','NOx',
           'CO','CO','CO','CO','CO',
           'HC','HC','HC','HC','HC',
           'PN','PN','PN','PN','PN'],
           ['-', '-',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','urban','extraurban','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','low','medium','high','extrahigh','150/d','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150',
           'complete','urban','rural','motorway','150']
         ]

tuples = list(zip(*arrays))

my_columns = pd.MultiIndex.from_tuples(tuples, names=['cycle', 'pollutant','phase'])

my_index = []

RegPollEmi = pd.DataFrame(index=my_index, columns=my_columns)
#RegPollEmi.columns

# Vehicle

## Lab

### Distance based

In [5]:
A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC_COLD*23*.' + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '150/d': [0,150],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLT*COLD*23*' + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '150/d': [0,150],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

In [6]:
def lab_A1(cycle, file, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = A1['cycle'][cycle]['phases'][phases][0]
        stop = A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=1, header=0)                          # read the proper sheets within each file
        print(time.time() - tic)
        
        v = df['DACtrl_ActualSpeed\n[kph]'][start:stop]                                  # speed (km/h)
        t_s = len(df.index[start:stop])                                                  # time(s)
        t_m = t_s/60                                                         # time (hours)
        t_h = t_s/3600                                                       # time (hours)
        v_mean = v.mean()                                                    # speed means
        d = v_mean * t_h                                                     # distance means
        
#         CO2_150 = df['c_CO2_Dil_g_vec\n[grams]'][0:150]
#         CO2_150d = CO2_150.sum()
        CO2 = df['c_CO2_Dil_g_vec\n[grams]'][start:stop]
        CO2_sum = CO2.sum()
        CO2km = CO2_sum/d                                                    # CO2 g/km 
        
        
        
        NOx = df['c_NOx_Dil_g_vec\n[grams]'][start:stop]
        NOx_sum = NOx.sum()*1000
        NOxkm = NOx_sum/d                                                    # NOx mg/km

        CO = df['c_CO_Dil_g_vec\n[grams]'][start:stop]
        CO_sum = CO.sum()*1000
        COkm = CO_sum/d                                                      # CO mg/km

        HC = df['c_THC_Dil_g_vec\n[grams]'][start:stop]
        HC_sum = HC.sum()*1000
        HCkm = HC_sum/d                                                      # HC mg/km

        flow_gas = df['c_Vmix_vec\n[m3/min]'][start:stop]                               # flow of gas mixture [m3/min]
        flow_avg = flow_gas.sum()/t_s                                       # average in m3/min
        vol_gas = flow_avg * t_m                                            # total volume of gas mixture (m3)
        vol_gas_cc = vol_gas * 1e6                                          # total volume of gas mixture (cm3)

        PN = df["Counter\n[cpcc]"][start:stop]                                         # column of PN flow in #/cm3
        try:
            PN_sum = PN.sum()                                                  
            PN_avg = PN_sum / t_s                                               # average of flow in #/cm3

            PN_total = PN_avg * vol_gas_cc                                      # total emitted in #
            PN_total_km = PN_total / d                                          # total emission in #/km
        except:
            PN_total_km = 0
        #key = each.split('T ')[1].split('.x')[0]                             # shortening the titles of the files

        #key = str(i) + "." + str(key)                                        # numbering the files
        dic_lab[each] = {
            'CO2km': CO2km,
            'NOxkm': NOxkm,
            'COkm': COkm,
            'HCkm': HCkm,
            'PNkm': PN_total_km,
            't_s': t_s, 
            'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

#### Alternative to handle vela8 files (NEDC Opel Astra)

In [ ]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


A1 = {
    'cycle': {
        'NEDC': { 'file': './' + path + '/*NEDC*COLD_23*.xlsx',# + filetype,             #  *FWD* for the Audi A3, C Cactus
                 'phases': {
                     '150/d': [0,150],
                     'urban': [0, 780],
                     'extraurban': [780, 1180], 
                     'complete': [0, 1180]}},
        'WLTP': { 'file': './' + path + '/*WLT*COLD*23*.xls',# + filetype,                 # WLTP_COLD for C Cactus
                 'phases': {
                     '150/d': [0,150],
                     'low': [0, 589],
                     'medium': [589,1022],
                     'high': [1022, 1477],
                     'extrahigh': [1477, 1800],
                     'complete': [0, 1800]}
                }, 
            },
    
     }

In [ ]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


def lab_A2(cycle, file, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = A1['cycle'][cycle]['phases'][phases][0]
        stop = A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=0, header=None)

        print(time.time() - tic)
        
        
        vals = df.values
        mux = pd.MultiIndex.from_arrays(df.ffill(1).values[:2, 1:], names=[None, 'Time'])   # trick to create the proper MultiIndex
        df = pd.DataFrame(df.values[2:, 1:], df.values[2:, 0], mux)                         # same here

        df.drop(df.index[[0,1]],inplace=True)                                               # remove the 2 first rows after the headers

        CO2 = df['Diluted grams']['CO2'][start:stop]
        CO2_sum = CO2.sum()
        NOx = df['Diluted grams']['NOx'][start:stop]
        NOx_sum = NOx.sum()*1000                                                            # change unit to mg/km
        CO = df['Diluted grams']['CO-L'][start:stop]
        CO_sum = CO.sum()*1000
        HC = df['Diluted grams']['HC'][start:stop]
        HC_sum = HC.sum()*1000                                                            # change unit to mg/km
               
        PN = df['CVS']['PN']
        PN.columns = ['PN_cm3', 'PN_s']
        PN = PN.PN_s[start:stop]
        PN_sum = PN.sum()

        v = df['Chassis dyno']['Speed'][start:stop]                                   # group speed by intervals                    
        v_mean = v.mean()                                                  # speed means
        t_s = len(df.index[start:stop])                                                #  time(s)
        t_m = t_s/60
        t_h = t_s/3600                                                                    # time (hours)

        d = v_mean * t_h #b.reset_index(drop=True) * d.reset_index(drop=True)                # distance means  

        CO2km = CO2_sum/d                                                    # CO2 per km 
        NOxkm = NOx_sum/d                                                    # NOx / km                                                     
        COkm = CO_sum/d                                                    # NOx / km                                                  
        HCkm = HC_sum/d
        PNkm = PN_sum/d
        
        dic_lab[each] = {
                'CO2km': CO2km,
                'NOxkm': NOxkm,
                'COkm': COkm,
                'HCkm': HCkm,
                'PNkm': PNkm,
                't_s': t_s, 
                'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

In [ ]:
dic_lab_all = {}

In [ ]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    for phase in A1['cycle'][cycle]['phases']:
        if cycle == 'NEDC':
            #print(A1['cycle'][cycle])
            dic_lab_all[cycle][phase] = lab_A2(cycle, file, phase) 
            #[phase] = run_A1(cycle, file, phase)
        else:
            dic_lab_all[cycle][phase] = lab_A1(cycle, file, phase) 

#### Follow from here

In [8]:
dic_lab_all = {}

In [9]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
        dic_lab_all[cycle][phase] = lab_A1(cycle, file, phase)

1.2058331966400146
1.085749626159668
1.063735008239746
1.0767443180084229
1.0467228889465332
1.0477235317230225
1.0607328414916992
1.2558677196502686
1.8122522830963135
1.250863790512085
1.0827479362487793
1.2518644332885742
1.08174729347229
1.0467238426208496
1.0457229614257812
1.0477230548858643
1.6701536178588867
1.6371309757232666
1.6581449508666992
1.728193998336792
1.673156499862671
1.599104881286621


In [10]:
a = pd.DataFrame.from_dict({(i,j,k): dic_lab_all[i][j][k] 
                           for i in dic_lab_all.keys() 
                           for j in dic_lab_all[i].keys()
                           for k in dic_lab_all[i][j].keys()},)
                       #orient='index')
a = a.replace(0, np.nan)
a

NEDC  \
                                                                     150/d   
      ././Porsche Cayenne\VELA2_02082017_001_Modal_NEDC_COLD_4WD_23°C.xlsx   
CO2km                                       3.389811e+02                     
COkm                                        1.595678e+03                     
HCkm                                        5.524804e+01                     
NOxkm                                       1.573968e+02                     
PNkm                                        1.953579e+10                     
d                                           6.426566e-01                     
t_s                                         1.500000e+02                     

                                                                            \
                                                                             
      ././Porsche Cayenne\VELA2_03082017_003_Modal_NEDC_COLD_4WD_23°C.xlsx   
CO2km                                       3.910322e+02                     
COkm                                        1.999848e+03                     
HCkm                                        5.297934e+01                     
NOxkm                                       2.291020e+02                     
PNkm                                        1.546279e+11                     
d                                           6.413034e-01                     
t_s                                         1.500000e+02                     

                                                                            \
                                                                             
      ././Porsche Cayenne\VELA2_04082017_003_Modal_NEDC_COLD_4WD_23°C.xlsx   
CO2km                                       3.307821e+02                     
COkm                                        1.376424e+03                     
HCkm                                        3.546331e+01                     
NOxkm                                       1.849598e+02                     
PNkm                                        5.027777e+11                     
d                                           6.505426e-01                     
t_s                                         1.500000e+02                     

                                                                            \
                                                                             
      ././Porsche Cayenne\VELA2_10082017_001_Modal_NEDC_COLD_4WD_23°C.xlsx   
CO2km                                       3.355281e+02                     
COkm                                        2.893252e+03                     
HCkm                                        7.797121e+01                     
NOxkm                                       1.686759e+02                     
PNkm                                        8.952420e+10                     
d                                           6.329954e-01                     
t_s                                         1.500000e+02                     

                                                                            \
                                                                  complete   
      ././Porsche Cayenne\VELA2_02082017_001_Modal_NEDC_COLD_4WD_23°C.xlsx   
CO2km                                       1.985495e+02                     
COkm                                        9.763418e+01                     
HCkm                                        4.708321e+00                     
NOxkm                                       3.211820e+01                     
PNkm                                        8.377083e+09                     
d                                           1.084391e+01                     
t_s                                         1.180000e+03                     

                                                                            \
                                                                             
      ./.

In [ ]:
# a.groupby(level=1).mean(axis=1)
a_avg =a.T.groupby(level=[0,1]).mean()       # mean of repeated tests
a_avg

In [ ]:
a_avg = a_avg.drop(['d','t_s'], axis=1)      # remove the columns not to be used
a_avg = a_avg.unstack(level=[-1,0])          # put the 2 rows in 1 row (larger)
a_avg = pd.DataFrame(a_avg).T
a_avg = a_avg.swaplevel(0,2,axis=1)          # order Multiindex levels
a_avg = a_avg.swaplevel(1,2,axis=1)          # order Multiindex levels
a_avg = a_avg.sort_index(axis=1, level=0)    # sort by pollutant
# a_avg[a_avg.columns.set_levels([['complete', 'urban','extraurban','150/d','complete',
#                                  'low','medium','high','extrahigh','150/d']], level=2)]
LAB1 = a_avg

In [ ]:
LAB1.index = [vehicle]                             # index name
LAB1.columns = LAB1.columns.droplevel(level=[0,1])    # no interested in Multiindex yet, so convert to regular df

LAB1.columns =['CO2_150/d_N','CO2_COMP_N','CO2_EU_N','CO2_URB_N',          # rename columns
               'CO_150/d_N','CO_COMP_N','CO_EU_N','CO_URB_N',
               'HC_150/d_N','HC_COMP_N','HC_EU_N','HC_URB_N',
               'NOx_150/d_N','NOx_COMP_N','NOx_EU_N','NOx_URB_N',
               'PN_150/d_N','PN_COMP_N','PN_EU_N','PN_URB_N',
               'CO2_150/d_W','CO2_COMP_W','CO2_EH_W','CO2_HIGH_W','CO2_LOW_W','CO2_MED_W',
               'CO_150/d_W','CO_COMP_W','CO_EH_W','CO_HIGH_W','CO_LOW_W','CO_MED_W',
               'HC_150/d_W','HC_COMP_W','HC_EH_W','HC_HIGH_W','HC_LOW_W','HC_MED_W',
               'NOx_150/d_W','NOx_COMP_W','NOx_EH_W','NOx_HIGH_W','NOx_LOW_W','NOx_MED_W',
               'PN_150/d_W','PN_COMP_W','PN_EH_W','PN_HIGH_W','PN_LOW_W','PN_MED_W']
              
LAB1 = LAB1[['CO2_COMP_N','CO2_URB_N','CO2_EU_N','CO2_150/d_N',             # reorder columns
            'NOx_COMP_N','NOx_URB_N','NOx_EU_N','NOx_150/d_N',
            'CO_COMP_N','CO_URB_N','CO_EU_N','CO_150/d_N',
            'HC_COMP_N','HC_URB_N','HC_EU_N','HC_150/d_N',
            'PN_COMP_N','PN_URB_N','PN_EU_N','PN_150/d_N',
            'CO2_COMP_W','CO2_LOW_W','CO2_MED_W','CO2_HIGH_W','CO2_EH_W','CO2_150/d_W',
            'NOx_COMP_W','NOx_LOW_W','NOx_MED_W','NOx_HIGH_W','NOx_EH_W','NOx_150/d_W',
            'CO_COMP_W','CO_LOW_W','CO_MED_W','CO_HIGH_W','CO_EH_W','CO_150/d_W',
            'HC_COMP_W','HC_LOW_W','HC_MED_W','HC_HIGH_W','HC_EH_W','HC_150/d_W',
            'PN_COMP_W','PN_LOW_W','PN_MED_W','PN_HIGH_W','PN_EH_W','PN_150/d_W']]

LAB1

### 150 sec (non-distance based)

In [ ]:
def lab_A1(cycle, file):#, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = 0 #A1['cycle'][cycle]['phases'][phases][0]
        stop = 150 #A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        df = pd.read_excel(each, sheetname=1, header=0)                          # read the proper sheets within each file
        print(time.time() - tic)
        
        v = df['DACtrl_ActualSpeed\n[kph]'][start:stop]                                  # speed (km/h)
        t_s = len(df.index[start:stop])                                                  # time(s)
        t_m = t_s/60                                                         # time (hours)
        t_h = t_s/3600                                                       # time (hours)
        v_mean = v.mean()                                                    # speed means
        d = v_mean * t_h                                                     # distance means
        
#         CO2_150 = df['c_CO2_Dil_g_vec\n[grams]'][0:150]
#         CO2_150d = CO2_150.sum()
        CO2 = df['c_CO2_Dil_g_vec\n[grams]'][start:stop]
        CO2_sum = CO2.sum()
        CO2km = CO2_sum/d                                                    # CO2 g/km 
        
        
        
        NOx = df['c_NOx_Dil_g_vec\n[grams]'][start:stop]
        NOx_sum = NOx.sum()*1000
        NOxkm = NOx_sum/d                                                    # NOx mg/km

        CO = df['c_CO_Dil_g_vec\n[grams]'][start:stop]
        CO_sum = CO.sum()*1000
        COkm = CO_sum/d                                                      # CO mg/km

        HC = df['c_THC_Dil_g_vec\n[grams]'][start:stop]
        HC_sum = HC.sum()*1000
        HCkm = HC_sum/d                                                      # HC mg/km

        flow_gas = df['c_Vmix_vec\n[m3/min]'][start:stop]                               # flow of gas mixture [m3/min]
        flow_avg = flow_gas.sum()/t_s                                       # average in m3/min
        vol_gas = flow_avg * t_m                                            # total volume of gas mixture (m3)
        vol_gas_cc = vol_gas * 1e6                                          # total volume of gas mixture (cm3)

        PN = df["Counter\n[cpcc]"][start:stop]                                         # column of PN flow in #/cm3
        try:
            PN_sum = PN.sum()                                                  
            PN_avg = PN_sum / t_s                                               # average of flow in #/cm3

            PN_total = PN_avg * vol_gas_cc                                      # total emitted in #
            PN_total_km = PN_total / d                                          # total emission in #/km
        except:
            PN_total = 0
            PN_total_km = 0

        #key = each.split('T ')[1].split('.x')[0]                             # shortening the titles of the files

        #key = str(i) + "." + str(key)                                        # numbering the files
        dic_lab[each] = {
            'CO2km': CO2_sum,
            'NOxkm': NOx_sum,
            'COkm': CO_sum,
            'HCkm': HC_sum,
            'PNkm': PN_total,
            't_s': t_s, 
            'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab

#### Only for Astra

In [ ]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
def lab_A2(cycle, file):#, phases): 
    files = glob.glob(file)
    dic_lab = {}                                                            # empty dictionary lab tests
    i = 1
    for each in files:
        
        start = 0 #A1['cycle'][cycle]['phases'][phases][0]
        stop = 150 #A1['cycle'][cycle]['phases'][phases][1]
        
        tic = time.time()
        
        df = pd.read_excel(each, sheetname=0, header=None)

        print(time.time() - tic)
        
        
        vals = df.values
        mux = pd.MultiIndex.from_arrays(df.ffill(1).values[:2, 1:], names=[None, 'Time'])   # trick to create the proper MultiIndex
        df = pd.DataFrame(df.values[2:, 1:], df.values[2:, 0], mux)                         # same here

        df.drop(df.index[[0,1]],inplace=True)                                               # remove the 2 first rows after the headers

        CO2 = df['Diluted grams']['CO2'][start:stop]
        CO2_sum = CO2.sum()
        NOx = df['Diluted grams']['NOx'][start:stop]
        NOx_sum = NOx.sum()*1000                                                            # change unit to mg/km
        CO = df['Diluted grams']['CO-L'][start:stop]
        CO_sum = CO.sum()*1000
        HC = df['Diluted grams']['HC'][start:stop]
        HC_sum = HC.sum()*1000                                                            # change unit to mg/km
               
        PN = df['CVS']['PN']
        PN.columns = ['PN_cm3', 'PN_s']
        PN = PN.PN_s[start:stop]
        PN_sum = PN.sum()
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""
        v = df['Chassis dyno']['Speed'][start:stop]                                   # group speed by intervals                    
        v_mean = v.mean()                                                  # speed means
        t_s = len(df.index[start:stop])                                                #  time(s)
        t_m = t_s/60
        t_h = t_s/3600                                                                    # time (hours)

        d = v_mean * t_h #b.reset_index(drop=True) * d.reset_index(drop=True)                # distance means  

        CO2km = CO2_sum/d                                                    # CO2 per km 
        NOxkm = NOx_sum/d                                                    # NOx / km                                                     
        COkm = CO_sum/d                                                    # NOx / km                                                  
        HCkm = HC_sum/d
        PNkm = PN_sum/d
        
        dic_lab[each] = {
                'CO2km': CO2_sum,
                'NOxkm': NOx_sum,
                'COkm': CO_sum,
                'HCkm': HC_sum,
                'PNkm': PN_sum,
                't_s': t_s, 
                'd': d}                                         # save means in the dictionary
        i = i+1
        
    return dic_lab
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""


In [ ]:
"""
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
ONLY FOR ASTRA!!
"""

for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
    if cycle == 'NEDC':
            #print(A1['cycle'][cycle])
        dic_lab_all[cycle] = lab_A2(cycle, file) 
            #[phase] = run_A1(cycle, file, phase)
    else:
        dic_lab_all[cycle] = lab_A1(cycle, file)

#### Follow from here

In [ ]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_lab_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_lab_all[cycle]= lab_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

In [ ]:
b = pd.DataFrame.from_dict({(i,j): dic_lab_all[i][j] 
                           for i in dic_lab_all.keys() 
                           for j in dic_lab_all[i].keys()},)
                       #orient='index')
b = b.replace(0,np.nan)
b

In [ ]:
b_avg = b.T.groupby(level=0).mean()
b_avg = b_avg.drop(['d','t_s'], axis=1)
b_avg

In [ ]:
NEDC150 = pd.DataFrame(b_avg.loc['NEDC',:])
NEDC150 = NEDC150.T
NEDC150.index = [vehicle]
WLTP150 = pd.DataFrame(b_avg.loc['WLTP',:])
WLTP150 = WLTP150.T
WLTP150.index = [vehicle]
WLTP150

LAB2 = pd.concat([NEDC150,WLTP150],axis=1)
LAB2.columns = ['CO2150_N','CO150_N','HC150_N','NOx150_N','PN150_N',
               'CO2150_W','CO150_W','HC150_W','NOx150_W','PN150_W']
LAB2 = LAB2[['CO2150_N','NOx150_N','CO150_N','HC150_N','PN150_N',
               'CO2150_W','NOx150_W','CO150_W','HC150_W','PN150_W']]
LAB2

In [ ]:
LAB = pd.concat([LAB1,LAB2],axis=1)
LAB = LAB[[
           'CO2_COMP_N','CO2_URB_N','CO2_EU_N','CO2_150/d_N','CO2150_N',             # reorder columns
           'NOx_COMP_N','NOx_URB_N','NOx_EU_N','NOx_150/d_N','NOx150_N',
           'CO_COMP_N','CO_URB_N','CO_EU_N','CO_150/d_N','CO150_N',
           'HC_COMP_N','HC_URB_N','HC_EU_N','HC_150/d_N','HC150_N',
           'PN_COMP_N','PN_URB_N','PN_EU_N','PN_150/d_N','PN150_N',
           'CO2_COMP_W','CO2_LOW_W','CO2_MED_W','CO2_HIGH_W','CO2_EH_W','CO2_150/d_W','CO2150_W',
           'NOx_COMP_W','NOx_LOW_W','NOx_MED_W','NOx_HIGH_W','NOx_EH_W','NOx_150/d_W','NOx150_W',
           'CO_COMP_W','CO_LOW_W','CO_MED_W','CO_HIGH_W','CO_EH_W','CO_150/d_W','CO150_W',
           'HC_COMP_W','HC_LOW_W','HC_MED_W','HC_HIGH_W','HC_EH_W','HC_150/d_W','HC150_W',
           'PN_COMP_W','PN_LOW_W','PN_MED_W','PN_HIGH_W','PN_EH_W','PN_150/d_W','PN150_W'
]]
LAB

## RDE

### Distance based

In [ ]:
A1 = {
    'cycle': {
        'Esperia': {'file': './'+path+'/*ESP*.xlsx'},
        'Labiena': {'file': './'+path+'/*LAB*.xlsx'}, 
             },
    
     }

In [ ]:
def road_A1(cycle, file):
    files_RDE = glob.glob(file)                                         # read all EMROAD files
    dic_RDE = {}                                                            # empty dictionary road test

    i = 1
    for each in files_RDE:

        tic = time.time()
        df = pd.read_excel(each, sheetname='TEST REPORT', header=6)             # read the proper sheets within each file
        print(time.time() - tic)
        
        df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)   # convert the headers to read them: ' ' for '_'
        df.rename(columns=lambda x: x.strip().replace(".", "_"), inplace=True)   # convert the headers to read them: '.' for '_'


        CO2_COMP = df.COMPLETE_TEST['CO2'][1]      # g/km
        CO2_URB = df.URBAN['CO2'][1]               # g/km
        CO2_RUR = df.RURAL['CO2'][1]               # g/km
        CO2_MOT = df.MOTORW_['CO2'][1]             # g/km
        #CO2 = [CO2_COMP,CO2_URB,CO2_RUR,CO2_MOT]

        NOx_COMP = df.COMPLETE_TEST['NOx'][1]*1000
        NOx_URB = df.URBAN['NOx'][1]*1000
        NOx_RUR = df.RURAL['NOx'][1]*1000
        NOx_MOT = df.MOTORW_['NOx'][1]*1000

        CO_COMP = df.COMPLETE_TEST['CO'][1]*1000      # g/km
        CO_URB = df.URBAN['CO'][1]*1000               # g/km
        CO_RUR = df.RURAL['CO'][1]*1000               # g/km
        CO_MOT = df.MOTORW_['CO'][1]*1000             # g/km

        HC_COMP = df.COMPLETE_TEST['THC'][1]*1000      # g/km
        HC_URB = df.URBAN['THC'][1]*1000               # g/km
        HC_RUR = df.RURAL['THC'][1]*1000               # g/km
        HC_MOT = df.MOTORW_['THC'][1]*1000             # g/km

        PN_COMP = df.COMPLETE_TEST['PN'][1]      # g/km
        PN_URB = df.URBAN['PN'][1]               # g/km
        PN_RUR = df.RURAL['PN'][1]               # g/km
        PN_MOT = df.MOTORW_['PN'][1]             # g/km

        #key = each.split('T ')[1].split('.x')[0]
        ###########################################       each = str(i) + "_" + str(each)
        dic_RDE[each] = {'CO2_COMP':CO2_COMP,'CO2_URB':CO2_URB,'CO2_RUR':CO2_RUR,'CO2_MOT':CO2_MOT,
                          'NOx_COMP':NOx_COMP,'NOx_URB':NOx_URB,'NOx_RUR':NOx_RUR,'NOx_MOT':NOx_MOT,
                          'CO_COMP':CO_COMP,'CO_URB':CO_URB,'CO_RUR':CO_RUR,'CO_MOT':CO_MOT,
                          'HC_COMP':HC_COMP,'HC_URB':HC_URB,'HC_RUR':HC_RUR,'HC_MOT':HC_MOT,
                          'PN_COMP':PN_COMP,'PN_URB':PN_URB,'PN_RUR':PN_RUR,'PN_MOT':PN_MOT,
                        }                                               # save means in the dictionary
        i = i+1
    return dic_RDE

In [ ]:
dic_road_all = {}

In [ ]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_road_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_road_all[cycle]= road_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

In [ ]:
c = pd.DataFrame.from_dict({(i,j): dic_road_all[i][j] 
                           for i in dic_road_all.keys() 
                           for j in dic_road_all[i].keys()},)
                       #orient='index')

c = c.replace(0,np.nan)                  # We put NaN instead of 0 so that those are not considered in the .mean()
c.T

In [ ]:
c_avg = c.T.groupby(level=0).mean()
c_avg = c_avg[['CO2_COMP','CO2_URB','CO2_RUR','CO2_MOT',
              'NOx_COMP','NOx_URB','NOx_RUR','NOx_MOT',
              'CO_COMP','CO_URB','CO_RUR','CO_MOT',
              'HC_COMP','HC_URB','HC_RUR','HC_MOT',
              'PN_COMP','PN_URB','PN_RUR','PN_MOT']]
c_avg

In [ ]:
RDE1 = pd.DataFrame(c_avg.mean())
RDE1.columns = [vehicle]
RDE1 = RDE1.T
RDE1

### RDE 150 sec (in g)

In [ ]:
def road_A1(cycle, file):
    files_RDE = glob.glob(file)                                         # read all EMROAD files
    dic_RDE = {}                                                            # empty dictionary road test

    i = 1
    for each in files_RDE:

        tic = time.time()
        df = pd.read_excel(each, sheetname='CALCULATED DATA', header=1,skiprows=[2])
        print(time.time() - tic)
        
        df.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)   # convert the headers to read them: ' ' for '_'
        df.rename(columns=lambda x: x.strip().replace(".", "_"), inplace=True)   # convert the headers to read them: '.' for '_'
    
        min_ind = df.loc[(df["Inst__Mass_CO2"] > 0.1)].index[0] # minimum index, engine turned on
           #  & (df["Inst__Mass_NOx"] > 0.000001)           # threshold chosen by looking the files
    
        CO2_150 = df.Inst__Mass_CO2[min_ind:min_ind+150].sum()           # grams of CO2
        NOx_150 = df.Inst__Mass_NOx[min_ind:min_ind+150].sum()*1000      # miligrams of NOx
        CO_150 = df.Inst__Mass_CO[min_ind:min_ind+150].sum()*1000       # miligrams of CO
        HC_150 = df.Inst__Mass_THC[min_ind:min_ind+150].sum()*1000      # miligrams of HC
        PN_150 = df.Inst__PN[min_ind:min_ind+150].sum()                 # total PN
        
        dic_RDE[each] = {
            'CO2 150': CO2_150,
            'NOx 150': NOx_150,
            'CO 150': CO_150,
            'HC 150': HC_150,
            'PN 150': PN_150
        }                                              # save means in the dictionary
        
        i = i+1
        
    return dic_RDE

In [ ]:
dic_road_all = {}

In [ ]:
for cycle in A1['cycle']:
    #print(cycle)
    dic_road_all[cycle] ={}
    file = A1['cycle'][cycle]['file']
    #for phase in A1['cycle'][cycle]['phases']:
        #print(A1['cycle'][cycle])
    dic_road_all[cycle]= road_A1(cycle, file)                          #[phase] = run_A1(cycle, file, phase)

In [ ]:
d = pd.DataFrame.from_dict({(i,j): dic_road_all[i][j] 
                           for i in dic_road_all.keys() 
                           for j in dic_road_all[i].keys()},)
                       #orient='index')
d = d.replace(0,np.nan)
d

In [ ]:
d_avg = d.T.groupby(level=0).mean()
d_avg = d_avg[['CO2 150','NOx 150','CO 150','HC 150','PN 150']]
d_avg

In [ ]:
e = pd.DataFrame(d_avg.mean())
e.columns = [vehicle]
RDE2 = e.T
RDE2

In [ ]:
RDE = pd.concat([RDE1,RDE2],axis=1)
RDE = RDE[['CO2_COMP','CO2_URB','CO2_RUR','CO2_MOT','CO2 150',
             'NOx_COMP','NOx_URB','NOx_RUR','NOx_MOT','NOx 150',
             'CO_COMP','CO_URB','CO_RUR','CO_MOT','CO 150',
             'HC_COMP','HC_URB','HC_RUR','HC_MOT','HC 150',
             'PN_COMP','PN_URB','PN_RUR','PN_MOT','PN 150']]
RDE

## Concat lab and road

In [ ]:
veh = pd.concat([LAB,RDE], axis=1)#, ignore_index=True)
veh.insert(0, "T_A", t_a)
veh.insert(0,"fuel",fuel)
# AUDI_A1.columns =db_regpoll.columns
#veh.drop('Peugeot Partner', inplace=True)
veh

In [ ]:
veh.columns = my_columns
RegPollEmi = RegPollEmi.append(veh)

In [ ]:
RegPollEmi                             #.sort_index()#.drop('Peugeot Partner', inplace=True)

# Excel and DataFrame

In [ ]:
RegPollEmi.to_excel('veh.xlsx')

In [2]:
dfr = pd.read_excel('veh.xlsx',header=(0,1,2))
dfr

cycle                fuel  T_A        NEDC                          \
pollutant               -    -         CO2                           
phase                   -    -    complete       urban  extraurban   
Audi A1          gasoline   98  115.010052  141.499340   99.544178   
BMW 530d           diesel  124  142.593498  183.514712  118.979382   
Audi A3            diesel  120  151.025535  177.388071  135.889201   
Citroen Cactus     diesel   95  114.342562  130.942092  104.667121   
Fiat 500x          diesel  144  191.497721  215.876307  177.539683   
Opel Astra       gasoline  103  130.524707  162.965505  111.592557   
Fiat Panda       gasoline  119  129.183095  169.412697  105.658152   
Kia Sportage       diesel  124  130.444277  157.244714  114.835875   
Peugeot Partner    diesel  112  116.523647  130.515602  108.385462   
Porsche Cayenne    diesel  179  197.610753  239.914162  173.322478   
Renault Twingo   gasoline  112  120.245945  155.692207   99.469361   
VW Golf            diesel  117  122.991381  152.002223  105.176443   

cycle                                                                       \
pollutant                                       NOx                          
phase                 150/d         150    complete      urban  extraurban   
Audi A1          211.819597  139.945888   11.489641  24.933212    3.640121   
BMW 530d         238.066331  154.864504   15.070664  20.150810   12.136080   
Audi A3          233.258413  149.479892   67.007396  66.526807   67.285514   
Citroen Cactus   167.571131  109.996445   29.155280  54.059993   14.636739   
Fiat 500x        276.482376  176.982248  196.017402  78.637758  263.499495   
Opel Astra       256.357918  169.924249   10.096388  25.069613    1.360553   
Fiat Panda       229.218358  151.089461    4.073624   7.087758    2.310302   
Kia Sportage     212.269976  138.810237   38.779593  54.625389   29.551978   
Peugeot Partner  162.998764  107.015247   13.569765  34.975806    1.119893   
Porsche Cayenne  349.080869  224.048574   41.832054  78.838226   20.582289   
Renault Twingo   200.637873  132.665124    4.597819   9.416676    1.773294   
VW Golf          213.664048  134.854343   17.106838  21.802389   14.144599   

cycle                ...           RDE                                         \
pollutant            ...            HC                                     PN   
phase                ...      complete urban rural motorway 150      complete   
Audi A1              ...           NaN   NaN   NaN      NaN NaN  1.148899e+12   
BMW 530d             ...           NaN   NaN   NaN      NaN NaN  7.359687e+10   
Audi A3              ...           NaN   NaN   NaN      NaN NaN           NaN   
Citroen Cactus       ...           NaN   NaN   NaN      NaN NaN           NaN   
Fiat 500x            ...           NaN   NaN   NaN      NaN NaN           NaN   
Opel Astra           ...           NaN   NaN   NaN      NaN NaN  2.361732e+12   
Fiat Panda           ...           NaN   NaN   NaN      NaN NaN  4.645039e+11   
Kia Sportage         ...           NaN   NaN   NaN      NaN NaN  4.955236e+09   
Peugeot Partner      ...           NaN   NaN   NaN      NaN NaN           NaN   
Porsche Cayenne      ...           NaN   NaN   NaN      NaN NaN           NaN   
Renault Twingo       ...           NaN   NaN   NaN      NaN NaN  6.668188e+10   
VW Golf              ...           NaN   NaN   NaN      NaN NaN           NaN   

cycle                                                                    
pollutant                                                                
phase                   urban         rural      motorway           150  
Audi A1          1.953222e+12  5.146943e+11  7.160030e+11  1.760148e+13  
BMW 530d         1.190758e+11  8.978261e+10  5.733847e+09  2.561064e+11  
Audi A3                   NaN           NaN           NaN           NaN  
Citroen Cactus            NaN           NaN           NaN           NaN  
Fiat 500x                 NaN           NaN 

## Extra stuff (testing)

In [3]:
RegPollEmi = dfr     # if values changed manually in the veh Excel, then run this to copy it into the dataframe
RegPollEmi.WLTP.PN['150']

Audi A1            5.766000e+12
BMW 530d           5.500000e+11
Audi A3            1.070000e+13
Citroen Cactus     3.214000e+10
Fiat 500x          3.767000e+11
Opel Astra         2.004559e+13
Fiat Panda         2.793954e+12
Kia Sportage       4.448571e+10
Peugeot Partner    4.320099e+10
Porsche Cayenne    1.078892e+11
Renault Twingo     5.995552e+12
VW Golf            3.644982e+12
Name: 150, dtype: float64

In [ ]:
dfr.WLTP.HC['high']